买点：  
macd金叉+kdj前面5天内有金叉，后一天起。
卖点：  
kdj死叉，后一天结束。

In [1]:
import pandas as pd
import numpy as np
import talib
# import tqdm
from tqdm.notebook import tqdm

%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15.0, 8.0) # 单位是inches

# 首先导入数据

In [2]:
data = pd.read_csv("../../数据获取/聚宽数据/最新数据.csv")
data=data.set_index(['Unnamed: 0', 'Unnamed: 1'])

In [35]:
data.head()

date   open   high    low  close
Unnamed: 0  Unnamed: 1                                        
000001.XSHE 0           2020-11-17  17.24  17.79  17.12  17.69
            1           2020-11-18  17.64  18.36  17.61  18.32
            2           2020-11-19  18.44  18.83  18.16  18.70
            3           2020-11-20  18.68  18.84  18.38  18.71
            4           2020-11-23  18.70  19.47  18.65  19.47

In [40]:
data_one = data.loc['000001.XSHE'].copy() 

In [41]:
data_one

,date,open,high,low,close
Unnamed: 1,,,,,
0,2020-11-17,17.24,17.79,17.12,17.69
1,2020-11-18,17.64,18.36,17.61,18.32
2,2020-11-19,18.44,18.83,18.16,18.70
3,2020-11-20,18.68,18.84,18.38,18.71
4,2020-11-23,18.70,19.47,18.65,19.47
...,...,...,...,...,...
360,2022-05-16,14.66,14.73,14.35,14.41
361,2022-05-17,14.51,14.72,14.32,14.71
362,2022-05-18,14.85,14.89,14.67,14.75


In [51]:
# 这里测试一个股票的
data_one = data.loc['000001.XSHE'].copy() # 这个是每一个股票的数据
# 首先添加macd的指标
_macd, _signal, _hist = talib.MACD(data_one['close'])
data_one.insert(
    loc=len(data_one.columns),
    column='macd',
    value=_hist # 我这里只是选取柱状图了。
)
# macd进行shift
data_one.insert(
    loc=len(data_one.columns),
    column='macd2',
    value=data_one['macd'].shift()
)
data_one.fillna(value=0) # 因为前面有很多None值，这里全部设置成0
# 然后金叉死叉的判断
data_one.insert(
    loc=len(data_one.columns),
    column='macd金叉',
    value= (data_one['macd'] > 0) & (data_one['macd2'] < 0)
)

# 这里判断一下是否有macd金叉的
data_one[data_one['macd金叉']]
# 其实并不多

,date,open,high,low,close,macd,macd2,macd金叉
Unnamed: 1,,,,,,,,
88,2021-03-29,21.11,21.48,21.03,21.32,0.004487,-0.043024,True
103,2021-04-20,20.92,21.78,20.76,21.52,0.019752,-0.051983,True
125,2021-05-25,23.54,24.73,23.45,24.60,0.017104,-0.038300,True
178,2021-08-09,17.91,19.25,17.84,19.06,0.059121,-0.063555,True
199,2021-09-07,18.60,19.56,18.35,19.24,0.045013,-0.036397,True
216,2021-10-11,19.00,19.74,19.00,19.40,0.072743,-0.018996,True
246,2021-11-22,18.03,18.25,17.90,18.12,0.006940,-0.007453,True
255,2021-12-03,17.64,17.70,17.41,17.65,0.004656,-0.007963,True
279,2022-01-07,17.10,17.28,17.06,17.20,0.011404,-0.016150,True


In [52]:
# 然后计算kdj金叉的
# 这里进行kdj的计算
_k, _d = talib.STOCH(
    data_one['high'],
    data_one['low'],
    data_one['close'],
    fastk_period=9,
    slowk_period=3,
    slowk_matype=0,
    slowd_period=3,
    slowd_matype=0)
data_one.insert(
    loc=len(data_one.columns),
    column='k',
    value= _k
)
data_one.insert(
    loc=len(data_one.columns),
    column='d',
    value= _d
)
# kd都平移
data_one.insert(
    loc=len(data_one.columns),
    column='k2',
    value=data_one['k'].shift()
)
data_one.insert(
    loc=len(data_one.columns),
    column='d2',
    value=data_one['d'].shift()
)
data_one.fillna(value=0) #  空值设置成0
# 然后这个kdj的金叉死叉
data_one.insert(
    loc=len(data_one.columns),
    column='kdj金叉',
    value= (data_one['k2'] < data_one['d2']) & (data_one['k'] > data_one['d'])
)
data_one.insert(
    loc=len(data_one.columns),
    column='kdj死叉',
    value= (data_one['k2'] > data_one['d2']) & (data_one['k'] < data_one['d'])
)
# 金叉要平移5天吧
for i in range(5):
    data_one.insert(
        loc=len(data_one.columns),
        column='kdj金叉' + str(i+2),
        value= data_one['kdj金叉'].shift(i+1)
    )
# 这里有一个汇总
data_one.fillna(value=False) # 空值设置成False
data_one.insert(
    loc=len(data_one.columns),
    column='kdj金叉汇总',
    value = data_one['kdj金叉'] | data_one['kdj金叉2'] | data_one['kdj金叉3'] | data_one['kdj金叉4']  | data_one['kdj金叉5']  | data_one['kdj金叉6']
)

In [53]:
data_one[data_one['kdj金叉汇总']]

,date,open,high,low,close,macd,macd2,macd金叉,k,d,k2,d2,kdj金叉,kdj死叉,kdj金叉2,kdj金叉3,kdj金叉4,kdj金叉5,kdj金叉6,kdj金叉汇总
Unnamed: 1,,,,,,,,,,,,,,,,,,,,
18,2020-12-11,18.56,18.65,18.08,18.38,NaN,NaN,False,9.465271,8.440718,7.492031,9.193055,True,False,False,False,False,False,False,True
19,2020-12-14,18.45,18.91,18.36,18.73,NaN,NaN,False,18.937987,11.965096,9.465271,8.440718,False,False,True,False,False,False,False,True
20,2020-12-15,18.65,18.70,18.01,18.59,NaN,NaN,False,27.682597,18.695285,18.937987,11.965096,False,False,False,True,False,False,False,True
21,2020-12-16,18.65,18.95,18.44,18.86,NaN,NaN,False,44.905810,30.508798,27.682597,18.695285,False,False,False,False,True,False,False,True
22,2020-12-17,18.76,18.92,18.45,18.80,NaN,NaN,False,54.410262,42.332890,44.905810,30.508798,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-05-16,14.66,14.73,14.35,14.41,-0.113344,-0.120197,False,20.237533,22.374660,23.626374,22.372391,False,True,False,True,False,False,False,True
361,2022-05-17,14.51,14.72,14.32,14.71,-0.082322,-0.113344,False,30.217852,24.693920,20.237533,22.374660,True,False,False,False,True,False,False,True
362,2022-05-18,14.85,14.89,14.67,14.75,-0.054181,-0.082322,False,40.632208,30.362531,30.217852,24.693920,False,False,True,False,False,True,False,True


In [57]:
data_one.loc[1, 'kdj金叉汇总']

False

# 多个股票的

In [62]:
income = [] # 收入放在这里边。这里边存放的是百分比。
with tqdm(range(len(data.index.levels[0]))) as bar:
    for i in data.index.levels[0]:
        data_one = data.loc[i].copy() # 这个是每一个股票的数据
        # 首先添加macd的指标
        _macd, _signal, _hist = talib.MACD(data_one['close'])
        data_one.insert(
            loc=len(data_one.columns),
            column='macd',
            value=_hist # 我这里只是选取柱状图了。
        )
        # macd进行shift
        data_one.insert(
            loc=len(data_one.columns),
            column='macd2',
            value=data_one['macd'].shift(fill_value=0)
        )
        data_one.fillna(value=0) # 因为前面有很多None值，这里全部设置成0
        # 然后金叉死叉的判断
        data_one.insert(
            loc=len(data_one.columns),
            column='macd金叉',
            value= (data_one['macd'] > 0) & (data_one['macd2'] < 0)
        )
        # 这里进行kdj的计算
        _k, _d = talib.STOCH(
            data_one['high'],
            data_one['low'],
            data_one['close'],
            fastk_period=9,
            slowk_period=3,
            slowk_matype=0,
            slowd_period=3,
            slowd_matype=0)
        data_one.insert(
            loc=len(data_one.columns),
            column='k',
            value= _k
        )
        data_one.insert(
            loc=len(data_one.columns),
            column='d',
            value= _d
        )
        # kd都平移
        data_one.insert(
            loc=len(data_one.columns),
            column='k2',
            value=data_one['k'].shift()
        )
        data_one.insert(
            loc=len(data_one.columns),
            column='d2',
            value=data_one['d'].shift()
        )
        data_one.fillna(value=0) #  空值设置成0
        # 然后这个kdj的金叉死叉
        data_one.insert(
            loc=len(data_one.columns),
            column='kdj金叉',
            value= (data_one['k2'] < data_one['d2']) & (data_one['k'] > data_one['d'])
        )
        data_one.insert(
            loc=len(data_one.columns),
            column='kdj死叉',
            value= (data_one['k2'] > data_one['d2']) & (data_one['k'] < data_one['d'])
        )
        # 金叉要平移5天吧
        for i in range(5):
            data_one.insert(
                loc=len(data_one.columns),
                column='kdj金叉' + str(i+2),
                value= data_one['kdj金叉'].shift(i+1)
            )
        # 这里有一个汇总
        data_one.fillna(value=False) # 空值设置成False
        data_one.insert(
            loc=len(data_one.columns),
            column='kdj金叉汇总',
            value = data_one['kdj金叉'] | data_one['kdj金叉2'] | data_one['kdj金叉3'] | data_one['kdj金叉4']  | data_one['kdj金叉5']  | data_one['kdj金叉6']
        )
        # 最后是计算啦
        j = 0
        while j < len(data_one):
            # 按照行遍历
            if data_one.loc[j, 'kdj金叉汇总'] and data_one.loc[j, 'macd金叉'] and i < len(data_one)-1:
                # 这里要开始计数了。
                # 这里要先判断一下是否
                j += 1
                # 超出边界就退出
                if j >= len(data_one):
                    break
                _close_start = data_one.loc[j, 'close'] # 可以当作买入价格，隔天的收盘价
                while j < len(data_one) and not data_one.loc[j, 'kdj死叉']:
                    # 直到遇到kdj死叉或者结束
                    j += 1
                # 隔一天。
                if j < len(data_one)-1:
                    j += 1
                # 不能超过限制
                if j >= len(data_one):
                    j = len(data_one)-1
                _close_end = data_one.loc[j, 'close'] # 可以当作卖出价格。
                income.append(
                    (_close_end-_close_start)/_close_start # 请注意，这里只是统计百分比，只有这个才有意义。
                )
            else:
                # 下一个。
                j += 1
        
        bar.update(1)
        
        

  0%|          | 0/4106 [00:00<?, ?it/s]

In [67]:
# 我这里有几个数据，首先全部的
print("全部数据:{}".format(len(income)))
income_up = [i for i in income if i > 0]
income_down = [i for i in income if i < 0]
income_up.sort()
income_down.sort()
print("上涨的{:.2f}%, 平均上涨：{:.2f}%".format(
    len(income_up)/len(income)*100,
    sum(income_up)/len(income_up)*100
    
))
print("下跌的{:.2f}%, 平均下跌：{:.2f}%".format(
    len(income_down)/len(income)*100,
    sum(income_down)/len(income_down)*100
    
))

print("最大上涨:{:.2f}%".format(max(income_up)*100))
print("最大下跌:{:.2f}%".format(min(income_down)*100))

print("上涨中位数:{:.2f}%".format(income_up[len(income_up)//2]*100))
print("下跌中位数:{:.2f}%".format(income_down[len(income_down)//2]*100))

全部数据:44683
上涨的42.29%, 平均上涨：4.70%
下跌的56.17%, 平均下跌：-3.52%
最大上涨:134.00
最大下跌:-70.95
上涨中位数:2.77
下跌中位数:-2.65


In [70]:
", ".join(["{:.2f}%".format(i*100) for i in income_down])

'-70.95%, -30.16%, -29.77%, -29.33%, -28.22%, -27.14%, -26.68%, -25.87%, -25.54%, -25.49%, -24.72%, -24.58%, -24.57%, -24.54%, -24.50%, -24.43%, -24.35%, -23.78%, -23.78%, -23.66%, -23.63%, -23.48%, -23.08%, -23.04%, -23.00%, -22.64%, -22.62%, -22.49%, -22.40%, -22.15%, -22.08%, -22.03%, -21.90%, -21.84%, -21.72%, -21.63%, -21.44%, -21.43%, -21.36%, -21.35%, -21.22%, -21.06%, -21.05%, -20.85%, -20.84%, -20.82%, -20.78%, -20.68%, -20.50%, -20.46%, -20.13%, -20.08%, -19.83%, -19.82%, -19.80%, -19.76%, -19.53%, -19.48%, -19.38%, -19.35%, -19.34%, -19.19%, -19.18%, -19.11%, -19.08%, -19.08%, -19.08%, -19.07%, -19.05%, -19.05%, -19.05%, -19.04%, -19.03%, -19.03%, -19.02%, -19.01%, -19.01%, -19.00%, -19.00%, -19.00%, -18.99%, -18.98%, -18.97%, -18.93%, -18.85%, -18.82%, -18.79%, -18.63%, -18.61%, -18.47%, -18.41%, -18.40%, -18.25%, -18.19%, -18.19%, -18.10%, -18.09%, -18.06%, -18.05%, -17.96%, -17.93%, -17.92%, -17.82%, -17.81%, -17.81%, -17.78%, -17.68%, -17.65%, -17.61%, -17.61%, -17.55%, 